In [12]:
import sys
sys.path.append("../")

import pandas as pd
import os
import numpy as np

from library import regulations
from library import analysis
from library import tables
import statsmodels.formula.api as smf


In [13]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
table_path = '/Users/kylieleblancKylie/domino/dofis/results/descriptive/'
data = pd.read_csv(os.path.join(data_path, 'clean', 'master_data_district.csv'),
                  sep=",")
data.sample(5)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,year,district,distname,distischarter,rating_academic,rating_financial,type,type_description,cntyname,...,district_status,type_urban,type_suburban,type_town,type_rural,eligible,teachers_nodegree,teachers_badegree,teachers_msdegree,teachers_phddegree
2904,2904,2014,91903,DENISON ISD,N,M,NaN,D,OTHER CENTRAL CITY SUBURBAN,GRAYSON,...,doi,0,1,0,0,1,0.005233,0.752382,0.236366,0.006019
1861,1861,2013,113903,LOVELADY ISD,N,M,NaN,H,RURAL,HOUSTON,...,doi,0,0,0,1,1,0.000000,0.790598,0.205128,0.000000
1466,1466,2013,50909,JONESBORO ISD,N,M,NaN,H,RURAL,CORYELL,...,doi,0,0,0,1,1,0.000000,0.797101,0.202899,0.000000
8018,8018,2018,137904,SANTA GERTRUDIS ISD,N,A,Pass,F,NON-METROPOLITAN FAST GROWING,KLEBERG,...,doi,0,0,1,0,1,0.000000,0.713701,0.286299,0.000000
6745,6745,2017,117901,BORGER ISD,N,M,Pass,G,NON-METROPOLITAN STABLE,HUTCHINSON,...,tps,0,0,1,0,1,0.005048,0.883897,0.111055,0.000000


In [3]:
print(data.students_white.hist())

AxesSubplot(0.125,0.11;0.775x0.77)


In [18]:
data = data[data.year == 2016]
data = data[data.doi == True]
data.sample(5)

,Unnamed: 0,year,district,distname,distischarter,rating_academic,rating_financial,type,type_description,cntyname,...,teachers_nodegree,teachers_badegree,teachers_msdegree,teachers_phddegree,bins_hispanic,bin20,bin40,bin60,bin80,bin100
5732,5732,2016,167902,MULLIN ISD,N,M,Pass,F,NON-METROPOLITAN FAST GROWING,MILLS,...,0.000000,0.768322,0.210402,0.023641,0.4,0,1,0,0,0
4910,4910,2016,3902,HUDSON ISD,N,M,Pass,G,NON-METROPOLITAN STABLE,ANGELINA,...,0.001035,0.812629,0.186335,0.000000,0.4,0,1,0,0,0
5995,5995,2016,227909,EANES ISD,N,M,Pass,B,MAJOR SUBURBAN,TRAVIS,...,0.017783,0.566126,0.410912,0.005180,0.2,1,0,0,0,0
6060,6060,2016,243905,WICHITA FALLS ISD,N,M,Pass,C,OTHER CENTRAL CITY,WICHITA,...,0.040928,0.745119,0.213061,0.000991,0.4,0,1,0,0,0
5830,5830,2016,186903,IRAAN-SHEFFIELD ISD,N,M,Pass,H,RURAL,PECOS,...,0.000000,0.789583,0.189583,0.020833,0.8,0,0,0,1,0


# Student Demographics

In [19]:
bins = [0, .2, .4, .6, .8,1]
labels = [.2, .4, .6,.8,1]
data['bins_hispanic']  = pd.cut(data['students_hisp'], bins, labels = labels)
data[['students_hisp', 'bins_hispanic']].sample(5)
data['bin20'] = np.where(data.bins_hispanic == .2, 1, 0)
data['bin40'] = np.where(data.bins_hispanic == .4, 1, 0)
data['bin60'] = np.where(data.bins_hispanic == .6, 1, 0)
data['bin80'] = np.where(data.bins_hispanic == .8, 1, 0)
data['bin100'] = np.where(data.bins_hispanic == 1, 1, 0)

In [20]:
len(data)

822

In [21]:
data.bins_hispanic.value_counts()

0.2    273
0.4    240
0.6    162
0.8     88
1.0     59
Name: bins_hispanic, dtype: int64

In [6]:
def create_count_proportion_df(data, list_of_regs):
    n_col = []
    p_20 = []
    p_40 = []
    p_60 = []
    p_80 = []
    p_100 = []
    reg_labels = []
    f_p = []

    for reg in list_of_regs:
        n_col.append((len(data[data[reg] == 1])))
        p_20.append(data[data.bins_hispanic == .2][reg].mean().round(2))
        p_40.append(data[data.bins_hispanic == .4][reg].mean().round(2))
        p_60.append(data[data.bins_hispanic == .6][reg].mean().round(2))
        p_80.append(data[data.bins_hispanic == .8][reg].mean().round(2))
        p_100.append(data[data.bins_hispanic == 1][reg].mean().round(2))
        reg_labels.append(regulations.labels[reg])
        formula = reg + '~ bin40 + bin60 + bin80 + bin100'
        df = data.dropna(subset=['students_hisp', reg])
        results = smf.ols(formula, data=df).fit()
        f_p.append(results.f_pvalue.round(2))

    df = pd.DataFrame(
            {'Regulation': reg_labels,
             'Count': n_col,
             '<20% Hispanic': p_20,
              '20% - 40%': p_40,
             '40% - 60%': p_60,
             '60% - 80%': p_80,
             '>80% Hispanic': p_100,
             'F-test p-value': f_p
             })
    return df

In [7]:
columns = ['Count', '<20% Hispanic', '20% - 40%', '40% - 60%', '60% - 80%', '>80% Hispanic', 'F-test p-value']
schedules_df = create_count_proportion_df(data, list_of_regs = regulations.schedules)
tables.df_to_excel(file = table_path + 'desc_exemptionsXdemographics.xlsx', df = schedules_df,
                  df_columns = columns, start_row = 4, start_col = 3)
schedules_df

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/statsmodels/compat/pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


,Regulation,Count,<20% Hispanic,20% - 40%,40% - 60%,60% - 80%,>80% Hispanic,F-test p-value
0,25.0811 - Minimum First Day of Instruction,798,0.98,0.98,0.97,0.97,0.90,0.01
1,25.081 - Minimum Minutes of Operation,356,0.37,0.48,0.48,0.50,0.31,0.01
2,25.0812 - Minimum Last Day of Instruction,231,0.27,0.26,0.31,0.33,0.27,0.68
3,25.082 - Pledge of Allegiance and Minute of Si...,208,0.25,0.29,0.24,0.30,0.10,0.04


In [8]:
class_size_df = create_count_proportion_df(data, list_of_regs = regulations.class_size)
tables.df_to_excel(file = table_path + 'desc_exemptionsXdemographics.xlsx', df = class_size_df,
                  df_columns = columns, start_row = 9, start_col = 3)
class_size_df

,Regulation,Count,<20% Hispanic,20% - 40%,40% - 60%,60% - 80%,>80% Hispanic,F-test p-value
0,25.112 - Class Size Maximum,363,0.48,0.42,0.46,0.40,0.39,0.52
1,25.113 - Notice of Class Size,299,0.41,0.36,0.37,0.27,0.31,0.18
2,25.111 - Maximum Student Teacher Ratio,127,0.14,0.15,0.20,0.17,0.08,0.27


In [9]:
certification_df = create_count_proportion_df(data, 
                                                 list_of_regs = regulations.certification)
tables.df_to_excel(file = table_path + 'desc_exemptionsXdemographics.xlsx', df = certification_df,
                  df_columns = columns, start_row = 13, start_col = 3)
certification_df

,Regulation,Count,<20% Hispanic,20% - 40%,40% - 60%,60% - 80%,>80% Hispanic,F-test p-value
0,21.003 - Teacher Certification Required,714,0.87,0.88,0.90,0.89,0.71,0.01
1,21.053 - Presentation of Teacher Certificates,240,0.28,0.28,0.35,0.27,0.27,0.58
2,21.057 - Notice of Uncertified Teacher,271,0.36,0.34,0.34,0.23,0.25,0.12


In [10]:
contracts_df = create_count_proportion_df(data, 
                                                 list_of_regs = regulations.contracts)
tables.df_to_excel(file = table_path + 'desc_exemptionsXdemographics.xlsx', df = contracts_df,
                  df_columns = columns, start_row = 17, start_col = 3)
contracts_df

,Regulation,Count,<20% Hispanic,20% - 40%,40% - 60%,60% - 80%,>80% Hispanic,F-test p-value
0,21.102 - Maximum Probationary Contract Length,419,0.58,0.52,0.48,0.47,0.31,0.00
1,21.401 - Minimum Service Days Required for Tea...,290,0.39,0.34,0.34,0.39,0.24,0.22
2,21.352 - Teacher Evaluation,166,0.21,0.22,0.15,0.22,0.22,0.46
3,21.354 - Administrator Evaluation,46,0.04,0.07,0.04,0.09,0.05,0.30


In [11]:
behavior_df = create_count_proportion_df(data, 
                                                 list_of_regs = regulations.behavior)
tables.df_to_excel(file = table_path + 'desc_exemptionsXdemographics.xlsx', df = behavior_df,
                  df_columns = columns, start_row = 22, start_col = 3)
behavior_df

,Regulation,Count,<20% Hispanic,20% - 40%,40% - 60%,60% - 80%,>80% Hispanic,F-test p-value
0,25.092 - Minimum Attendance for Class Credit,214,0.16,0.32,0.31,0.27,0.32,0.00
1,37.0012 - Designation of Campus Behavior Coord...,178,0.17,0.24,0.24,0.28,0.17,0.11
2,25.036 - Transfers,178,0.26,0.22,0.20,0.16,0.14,0.13
